In [929]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [930]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "PESEL",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [931]:
employees_df

,id,imię,nazwisko,PESEL,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Ewelina,Czarnowska,76052471848,Poznań,ul. Wapienna 94/9,48488382693,ewelina.czarnowska@kuba.pl,2014-01-01,None,menadżer,64.3
1,2,Janusz,Grygorowicz,83010511713,Wrocław,ul. Świerkowa 80/16,48461290165,janusz.grygorowicz@kuba.pl,2014-01-01,None,sprzedawca,45.9
2,3,Paulina,Zygmuntowicz,60012154948,Międzyrzecz,ul. Awaryjna 86/44,48952391475,paulina.zygmuntowicz@kuba.pl,2014-01-01,2015-05-31,sprzedawca,56.3
4,4,Justyna,Winiarska,62040740065,Wrocław,ul. Żegiestowska 1/2,48050219288,justyna.winiarska@kuba.pl,2014-01-01,None,mechanik,55.4
12,5,Urszula,Kiedrowicz,88091426181,Opole,ul. Stefana Batorego 69/19,48494955528,urszula.kiedrowicz@kuba.pl,2014-01-01,None,informatyk,46.5
3,6,Marlena,Szustakiewicz,68032329441,Wrocław,ul. Ogrodowa 59,48397033321,marlena.szustakiewicz@kuba.pl,2014-09-09,None,sprzedawca,53.0
5,7,Urszula,Kosalewicz,88021233920,Wrocław,ul. Kościuszki 49,48429164546,urszula.kosalewicz@kuba.pl,2015-07-29,None,mechanik,53.1
6,8,Justyna,Miller,94100604985,Prudnik,ul. Mostowa 26,48258199186,justyna.miller@kuba.pl,2015-11-19,None,mechanik,48.9
7,9,Krystian,Ekiert,72123075739,Wrocław,ul. Okrężna 82/39,48954231107,krystian.ekiert@kuba.pl,2016-02-17,None,pomocnik mechanika,35.3
8,10,Jacek,Kuczer,86092064256,Wrocław,ul. ks. Ignacego Skorupki 63/23,48256549226,jacek.kuczer@kuba.pl,2016-05-02,None,pomocnik mechanika,38.3


In [932]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [933]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [934]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [935]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4])-19 for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()
not_fixed_our_cars = our_cars[:]

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if len(not_fixed_our_cars) >= 1 and np.random.rand() <= weights[1]: #fixing car we currently have
           client_ids[i] = 0
           car_ids[i] = np.random.choice(not_fixed_our_cars)
           not_fixed_our_cars.remove(car_ids[i])
        elif np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [936]:
len(year_ceil), len(pd.unique(client_ids))

(1414, 1415)

In [937]:
year_ceil

[1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [938]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [939]:
len(buy_indexes[0]), len(sell_indexes[0]), len(service_indexes[0])

(771, 765, 486)

In [940]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [941]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [942]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [943]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [944]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])

menager = employees_df.iloc[0]
if int(menager["PESEL"][-2]) % 2 == 1:
    menager_sex = "M"
else:
    menager_sex = "K"
clients_df.loc[0] = [0, menager["imię"], menager["nazwisko"], menager_sex, menager["PESEL"], menager["miasto"], menager["adres"], False, menager["telefon"], menager["email"]]

unique_clients = np.unique(client_ids)
unique_clients = unique_clients[unique_clients != 0]
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Ewelina,Czarnowska,K,76052471848,Poznań,ul. Wapienna 94/9,False,48488382693,ewelina.czarnowska@kuba.pl
1,1,Lidia,Sydow,K,76112715691,Wrocław,ul. Huberta Wagnera 36/38,False,48404030222,s_@wp.pl
2,2,Barbara,Hnibieda,K,61091182439,Wrocław,ul. Piekarska 1/46,False,48437109502,hnibieda3@gmail.com
3,3,Weiwei,Górski,M,84083182916,Białystok,ul. Promyka 44,False,48702231836,wgorski84@gmail.com
4,4,Hanna,Wrzosek,K,64032473579,Wrocław,ul. Eugeniusza Kwiatkowskiego 34/19,False,48445047397,wrzose-1964@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1410,1410,Kamil,Szozda,M,80032367153,Wrocław,ul. Kwiatowa 97/28,False,48940337636,kamszozd1980@gmail.com
1411,1411,Marcin,Walkowiak,M,88011270270,Wrocław,ul. Makowa 80,False,48261813230,mar_walkowia1988@o2.pl
1412,1412,Marcin,Lis,M,75010863532,Wrocław,ul. Niezapominajki 93/30,False,48951142927,m.lis@gmail.com
1413,1413,Magdalena,Mroczkowska,K,None,Wrocław,ul. Staropolska 85/19,False,48333947306,m_mr@outlook.com


In [945]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [946]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Ewelina,Czarnowska,K,76052471848,Poznań,ul. Wapienna 94/9,False,48488382693,ewelina.czarnowska@kuba.pl
1,1,Lidia,Sydow,K,76112715691,Wrocław,ul. Huberta Wagnera 36/38,False,48404030222,s_@wp.pl
2,2,Barbara,Hnibieda,K,61091182439,Wrocław,ul. Piekarska 1/46,False,48437109502,hnibieda3@gmail.com
3,3,Weiwei,Górski,M,84083182916,Białystok,ul. Promyka 44,False,48702231836,wgorski84@gmail.com
4,4,Hanna,Wrzosek,K,64032473579,Wrocław,ul. Eugeniusza Kwiatkowskiego 34/19,False,48445047397,wrzose-1964@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1410,1410,Kamil,Szozda,M,80032367153,Wrocław,ul. Kwiatowa 97/28,False,48940337636,kamszozd1980@gmail.com
1411,1411,Marcin,Walkowiak,M,88011270270,Wrocław,ul. Makowa 80,False,48261813230,mar_walkowia1988@o2.pl
1412,1412,Marcin,Lis,M,75010863532,Wrocław,ul. Niezapominajki 93/30,False,48951142927,m.lis@gmail.com
1413,1413,Magdalena,Mroczkowska,K,None,Wrocław,ul. Staropolska 85/19,False,48333947306,m_mr@outlook.com


In [947]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [948]:
np.sum(clients_df['karta_lojalnościowa'])

76

In [949]:
np.sum(pd.isna(clients_df["PESEL"]))

193

# auta

In [950]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [951]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [952]:
df_samochody = create_car_table(max(car_ids))

In [953]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Mazda,3,4,2007,Hatchback,False,szary,Benzyna,1.8,116,Manualna,3,5,False
1,2,Bajaj,Discover 100 T,2,2002,,False,szary,Benzyna,0.7,108,Manualna,0,1,False
2,3,Ford,S-Max,4,2001,SUV,False,czarny,Diesel,2.5,163,Automatyczna,5,5,False
3,4,Ford,Fiesta,4,2004,Sedan,False,biały,Diesel,0.9,60,Manualna,3,5,False
4,5,Suzuki,Alto,4,2008,SUV,False,biały,Benzyna,1.1,68,Manualna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,1004,SEAT,Ateca,4,2005,Hatchback,False,czarny,Diesel,2.4,150,Manualna,5,5,False
1004,1005,Nissan,Primastar,4,2002,Sedan,False,czarny,Diesel,1.6,114,Manualna,3,5,True
1005,1006,BMW,X1,4,2004,SUV,False,czarny,Diesel,2.9,190,Automatyczna,5,2,True
1006,1007,Ford,Focus,4,2003,Sedan,True,biały,Benzyna,1.9,120,Manualna,5,5,False


In [954]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [955]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Manualna', 'Automatyczna', 'Półautomatyczna'], dtype=object)

# Usługi

In [956]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [957]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [958]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [959]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2006' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [960]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [961]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,2,2014-01-02,43300.0
1,2,2,2,2,2014-01-02,30000.0
2,3,3,3,3,2014-01-05,42300.0
3,4,4,4,3,2014-01-05,14500.0
4,5,5,5,2,2014-01-05,20900.0
...,...,...,...,...,...,...
766,767,727,1066,3,2017-12-19,28800.0
767,768,1005,1409,2,2017-12-20,32500.0
768,769,1006,1411,2,2017-12-23,56600.0
769,770,1008,1414,3,2017-12-30,18000.0


In [962]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [963]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [964]:
loyalty_card_customer_ids = [el[0] for el in loyalty_card_dates]
loyalty_card_dates_sep = [el[1] for el in loyalty_card_dates]

In [965]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:
        date_index = loyalty_card_customer_ids.index(customer_id)
        if loyalty_card_dates_sep[date_index] < df_sell.loc()[i]['data_zaksięgowania']:
            sell_prices[i] *= 0.97
            #print(i)

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
#print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\1822833189.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:


In [966]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,9,21,6,53800.0,1,2014-01-12
1,2,16,22,3,36300.0,1,2014-01-12
2,3,6,23,6,71100.0,1,2014-01-14
3,4,14,24,6,59042.0,18,2014-01-14
4,5,5,25,6,29700.0,1,2014-01-14
...,...,...,...,...,...,...,...
760,761,721,504,2,49900.0,1,2017-12-15
761,762,998,71,6,167280.0,6,2017-12-19
762,763,993,1410,2,23766.0,6,2017-12-21
763,764,996,1253,3,6834.0,6,2017-12-28


In [967]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

0

In [968]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [969]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(12, array([60, 52], dtype=int64)), (22, array([27], dtype=int64)), (24, array([36, 16], dtype=int64)), (25, array([23], dtype=int64)), (28, array([11], dtype=int64)), (30, array([26, 63], dtype=int64)), (32, array([52], dtype=int64)), (33, array([31], dtype=int64)), (34, array([8], dtype=int64)), (35, array([32,  9], dtype=int64)), (39, array([28, 25, 42], dtype=int64)), (41, array([58], dtype=int64)), (42, array([54], dtype=int64)), (47, array([ 2, 56], dtype=int64)), (50, array([55,  5], dtype=int64)), (57, array([11], dtype=int64)), (60, array([18], dtype=int64)), (61, array([5], dtype=int64)), (64, array([34, 35], dtype=int64)), (66, array([44, 54], dtype=int64)), (70, array([ 7, 21, 22], dtype=int64)), (71, array([48], dtype=int64)), (72, array([63], dtype=int64)), (74, array([10], dtype=int64)), (78, array([31], dtype=int64)), (80, array([39, 21, 38], dtype=int64)), (82, array([60, 62, 43], dtype=int64)), (83, array([34], dtype=int64)), (84, array([29], dtype=int64)), (86, arra

In [970]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [971]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Dźwignia zmiany biegów,Toyota Auris,112,1,1
1,Zbiornik paliwa,Toyota Auris,337,1,1
2,Rozrusznik,Toyota Yaris,445,2,1
3,Klocek hamulcowy,Renault Clio,17,3,4
4,Głośnik,Renault Clio,91,3,1
...,...,...,...,...,...
722,Dywan,SEAT Ateca,28,484,2
723,Hamulec ręczny,SEAT Ateca,58,484,1
724,Zawór EGR,,500,485,1
725,Pompa paliwowa,Hero Passion XPro,24,485,1


# Wyposażenie

In [972]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [973]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Dźwignia zmiany biegów,Toyota Auris,112.0
1,2,Zbiornik paliwa,Toyota Auris,337.0
2,3,Rozrusznik,Toyota Yaris,445.0
3,4,Klocek hamulcowy,Renault Clio,17.0
4,5,Głośnik,Renault Clio,91.0
...,...,...,...,...
721,528,Oświetlenie boczne,Hero Honda CD Deluxe,28.0
722,529,Dywan,SEAT Ateca,28.0
723,530,Hamulec ręczny,SEAT Ateca,58.0
725,531,Pompa paliwowa,Hero Passion XPro,24.0


# Usługi - uzupełnienie

In [974]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [975]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,12,0,7,2014-01-26,530
1,2,22,43,4,2014-01-28,530
2,3,24,46,7,2014-01-30,190
3,4,25,47,8,2014-01-31,20
4,5,28,51,8,2014-02-01,1080
...,...,...,...,...,...,...
481,482,1001,1405,4,2017-12-15,10
482,483,1003,1407,7,2017-12-16,760
483,484,1004,1408,8,2017-12-19,130
484,485,996,0,4,2017-12-20,620


In [976]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [977]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [978]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,1,2,1
2,3,2,3,1
3,4,3,4,4
4,5,3,5,1
...,...,...,...,...
722,723,484,529,2
723,724,484,530,1
724,725,485,95,1
725,726,485,531,1


In [979]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [980]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))[0]
    service_date = get_service_date(part_id)
    
    row = [int(i+1), int(part_id), total_cost, int(quantity)]
    parts_purchase_pd.loc[i] = row


In [981]:
parts_purchase_pd

,id,id_części,koszt,ilość
0,1.0,1.0,525.0,6.0
1,2.0,2.0,2214.0,7.0
2,3.0,3.0,2406.0,6.0
3,4.0,4.0,59.0,4.0
4,5.0,5.0,403.0,5.0
...,...,...,...,...
527,528.0,528.0,68.0,3.0
528,529.0,529.0,75.0,3.0
529,530.0,530.0,317.0,6.0
530,531.0,531.0,108.0,6.0


In [982]:
parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [983]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [984]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)